In [1]:
!pip install --upgrade pip
!pip install openai-whisper fastapi uvicorn python-multipart ffmpeg-python nest_asyncio pyngrok
!sudo apt update && sudo apt install ffmpeg -y



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 137.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 150.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.6 MB/s eta 0:00:00
 

In [2]:
import whisper
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import FileResponse
import uuid, os
import uvicorn
import nest_asyncio

app = FastAPI()
nest_asyncio.apply()

UPLOAD_DIR = "uploads"
SRT_DIR = "srt_outputs"
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(SRT_DIR, exist_ok=True)

model = whisper.load_model("base")  # İstersen "small", "medium", "large"

def format_timestamp(seconds: float) -> str:
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    ms = int((seconds - int(seconds)) * 1000)
    return f"{h:02}:{m:02}:{s:02},{ms:03}"

@app.post("/upload")
async def upload_video(file: UploadFile = File(...)):
    file_id = str(uuid.uuid4())
    input_path = os.path.join(UPLOAD_DIR, f"{file_id}_{file.filename}")
    output_path = os.path.join(SRT_DIR, f"{file_id}.srt")

    with open(input_path, "wb") as f:
        f.write(await file.read())

    result = model.transcribe(input_path, fp16=True)

    with open(output_path, "w", encoding="utf-8") as f:
        for i, segment in enumerate(result["segments"]):
            f.write(f"{i+1}\n")
            f.write(f"{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}\n")
            f.write(f"{segment['text'].strip()}\n\n")

    return {"message": "SRT hazır", "srt_file": f"/download/{os.path.basename(output_path)}"}

@app.get("/download/{filename}")
async def download_srt(filename: str):
    path = os.path.join(SRT_DIR, filename)
    return FileResponse(path, filename=filename, media_type="application/x-subrip")


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 154MiB/s]


In [3]:
from pyngrok import ngrok

# 🔑 NGROK TOKENINI BURAYA YAZ
ngrok.set_auth_token("NGROK_AUTH_TOKENIN")

# 🌐 Bağlantıyı aç
public_url = ngrok.connect(7860)
print("🚀 Sunucunuz burada çalışıyor:", public_url)

# 🔥 Uvicorn ile sunucuyu başlat
uvicorn.run(app, host="0.0.0.0", port=7860)



ERROR:  authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.
ERROR:  Your authtoken: NGROK_AUTH_TOKENIN
ERROR:  Instructions to install your authtoken are on your ngrok dashboard:
ERROR:  https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_105
ERROR:  https://ngrok.com/docs/errors/err_ngrok_105
ERROR:  


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: NGROK_AUTH_TOKENIN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.